## Final Project E-Commerce: Product Range Analysis (Code)

### Student: Yasmin Madjitey

Project Goal:
The main goal here is to analyze the store's product range with regards to profitability and customer demand.
That is, a careful analysis to provide the basis for targeted adjustments to boost sales, hence profits.

`Link to presentation: https://docs.google.com/presentation/d/1AZO10liTidFQEU5HG9tAtl0utLLmsi3q7SZSwmW-c70/edit?usp=sharing`

### Task 1

### Carry out exploratory data analysis:

In [ ]:
# import necessary libraries
from zipfile import ZipFile
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import networkx as nx
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
# specifying the zip file name where the dataset is stored
file_name = "ecommerce_dataset_us.zip"
  
# opening the zip file in 'read' mode
with ZipFile(file_name) as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    zip.extractall()

In [ ]:
# open the extracted csv file as a pandas dataframe
df_original = pd.read_csv("ecommerce_dataset_us.csv", sep='\t')

In [ ]:
# a glance at the data
df_original

#### Data preprocessing

In [ ]:
# set a copy of the original dataset to df
df = df_original.copy()

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe(include = 'all')

Note: 
- customer ID column is of type float instead of an object since aggregations cannot be done on customer ID. 
- Invoice date needs to be changed from object type to datetime.
- customer id contains nan values
- Quantity column has negative values
- Price column has negative values

In [ ]:
df.tail()

In [ ]:
df.sort_values('UnitPrice')

In [ ]:
# price should be > than 0
df.loc[df['UnitPrice'] <= 0]

Note:
- These products seem to be damaged products which cannot be sold, hence they have no customer ids and cost nothing (0.0 unit price) as well

In [ ]:
df.loc[df['UnitPrice'] <= 0].sample(50)

Note:
- realize that majority of the 0.00 price values have corresponding 'nan' customer id values, 'nan' descriptions and (-) quantity values
- these could pass as damaged goods recorded by the store with no price values.

In [ ]:
# convert the 0.00 priced values to nan values and visualize missingness in dataset
df.loc[df['UnitPrice'] <= 0, 'UnitPrice'] = np.nan
# test
print(df.loc[df['UnitPrice'].isnull()])

# to visualize data completeness (missing values)
%matplotlib inline
msno.matrix(df)

Note:

- they represent a very small fraction of the dataset, hence it is safe to remove them.

In [ ]:
# remove these values since they would not be valuable to the analysis
df = df.loc[~df['UnitPrice'].isnull()]
# verify if the values have been correctly removed
df.info()
df.loc[df['UnitPrice'].isnull()]

In [ ]:
# finding outliers by price
outliers = df.groupby('Description').agg({'UnitPrice': 'mean'}).sort_values('UnitPrice', ascending = False)
outliers.head(20)

In [ ]:
outliers.tail(20)

Note:

- Descriptions such as Adjust bad debt, AMAZON FEE, CRUK Commission, DOTCOM POSTAGE, Manual, SAMPLES, Bank Charges have average prices larger than the average unit price of the products. They do not seem to represent store products like the rest do. Furthermore, these items are not direct indicators of orders and might skew the sales distribution. They do not directly tie to individual product order, hence, they would be removed.

In [ ]:
remove_items = ['DOTCOM POSTAGE', 'Adjust bad debt', 'CRUK Commission', 'Bank Charges', 'AMAZON FEE', 'Manual',
                'POSTAGE', 'SAMPLES']

df = df.loc[~df['Description'].isin(remove_items)]

#test
df.loc[df['Description'].isin(remove_items)]

In [ ]:
df.loc[df['Quantity'] <= 0]

Note:
- the negative quantity values with invoice numbers starting with 'C' look like returned or cancelled goods especially because they have corresponding customer ids.

- Discount on the otherhand affect prices.

- It is a great idea to get and remove both the original and returned items from the customers' orders.

- Another way would be to keep them since running a .sum() on quantity would cancel out these items.

- Let's see if there are some without customer ids first

In [ ]:
df.loc[(df['Quantity'] <= 0) & (df['CustomerID'].isnull())]

In [ ]:
df.loc[(df['Quantity'] <= 0) & (df['CustomerID'].isnull())].sample(50)

Note:
- We do have cancelled or returned items without customer ids. One thing about these items is that they do represent store items. Some of the questions that come to mind include; were these items cancelled way before they were paid for by the customer, hence they do not have customer ids? were they probably cancelled by the store? or left in the customers' shopping cart until the alloted time lapsed?


- Although these values constitute a small fraction of the dataset, they would not be removed since more information is needed to clear doubts. 

- Also because they represent losses to the store. removing them without justification would overstate the store's actual sales.

In [ ]:
df.loc[df['Quantity'] <= 0]

In [ ]:
df.loc[df.InvoiceNo.str.startswith('C')]

- We see that all the items with negative quantity values have the invoice number starting with 'C'. We can conclude that, all the negative quantities in our dataset are cancelled or returned items except for discount

- Before moving on, let's visualize common returned items

In [ ]:
returned_df = df.loc[df.InvoiceNo.str.startswith('C')]

grouped = returned_df.groupby('Description')['InvoiceNo'].nunique().sort_values(ascending = False)
print(grouped.head())

fig = px.bar(data_frame = grouped.head(20))
fig.update_layout(title = 'Frequently Returned or Cancelled Items')
fig.show()

Note:
- 'REGENCY CAKESTAND 3 TIER' is the most returned store item.

- 'Discount' in the dataset represents dicounts applied on products price (reduces product price), hence, the negative value. These would not be removed since it affects the price of items.

In [ ]:
df[df['Description'] == 'Discount']

In [ ]:
# get the pair of original and cancelled or returned orders
#cancelled_orders = df.loc[df.groupby(['CustomerID', 'StockCode'])['Quantity'].transform('sum').eq(0)].sort_values(by=['CustomerID', 'StockCode'])
#print(cancelled_orders)

# drop these items:
#get_index
#get_index = cancelled_orders.index

#df = df.drop(index = get_index)
#df

Note:

- On a second thought, removing returned and the corresponding original orders is quiet tricky. What if discounts were applied on these items? removing them without their corresponding discounts would render the analysis biased.

- It is also not clear how the store's discount policy works.

- It is safer to leave them in the dataset since running a .sum() would cancel them out.

In [ ]:
df

In [ ]:
# checking for missing values in customer id column
df['CustomerID'].isnull().sum()

Note:
- I would not remove the nan values in the customer id column as it is a big amount of data.

In [ ]:
# converting invoice date from object type to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# converting customer ID column to object datatype
df['CustomerID'] = df.CustomerID.astype('str')

# test
print(df.info())
print(df.head())

In [ ]:
# remove .0 at the end of the customer ID
df['CustomerID'] = df.CustomerID.str[:-2]
df.head(10)

In [ ]:
df.CustomerID.value_counts()

In [ ]:
df.loc[df['CustomerID'] == 'n', 'CustomerID'] = np.nan

# test
df[df['CustomerID'].isnull()]

#### Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
# timeframe of data
print("Time From:")
print(str(df.InvoiceDate.min()))
print("To:")
print(str(df.InvoiceDate.max()))
print('\n')
print("A year's data")

In [ ]:
# create a sales column (quantity * unitprice)
df['Sales'] = df['Quantity'] * df['UnitPrice']
df

***Univariate analysis***

In [ ]:
print(df.Sales.describe())

# plotting sales distribution before applying transformation
df['Sales'].plot.hist(bins = 50, alpha = 0.5)
plt.xlim(-10000,  10000)
plt.title('Distribution of Sales; Untransformed Values');

Note:

- the minimum value in the sales data is '-168469.600000' and maximum is '168469.600000'. This must be a returned or cancelled item.

In [ ]:
# plotting sales distribution after applying cuberoot transformation because of negative values in the dataset
sales_cuberoot = np.cbrt(df['Sales'])
sales_cuberoot.plot.hist(bins = 200, alpha = 0.5)
plt.xlim(-5, 15)
plt.title('Distribution of Sales: Cuberoot Transformed Values');

Note:
- Cuberoot transformations reduce or remove skewness of a dataset especially where negative values are involved.

- we see that the sales column has outliers. Since the store distributes on wholesale terms as well, these values are accounted for.

***Bivariate Analysis***

In [ ]:
# Daily unique orders
number_purchases_per_day = df.set_index('InvoiceDate').resample('D').nunique().reset_index()
#print(number_purchases_per_day)

fig = px.line(data_frame = number_purchases_per_day, x = 'InvoiceDate', y = 'InvoiceNo')
fig.update_layout(title = "Daily Unique or New Orders")
fig.show()

Note:
- From the graph, we can see that there are days where the store had 0 number of orders. Why is that?

In [ ]:
number_purchases_per_day.loc[number_purchases_per_day['Quantity']== 0]

Note:
- the dates here are additive by 7, could that replicate the last day of the week (Sunday) where most stores are closed?

In [ ]:
# total purchases and sales per day
total_purch_per_day = df.set_index('InvoiceDate').resample('D').sum().reset_index()
print(total_purch_per_day)

fig1 = px.line(data_frame = total_purch_per_day, x = 'InvoiceDate', y = 'Quantity')
fig1.update_layout(title = "Total Purchases per Day")
fig1.show()

fig2 = px.line(data_frame = total_purch_per_day, x = 'InvoiceDate', y = 'Sales')
fig2.update_layout(title = "Total Sales per Day")
fig2.show()

Note:

- We see that the store had big sales on Nov. 12 & Sep. 18, whereas the store had its biggest orders on these days including Oct. 3, which orders surpassed that on Sep. 18.

- Larger quatities sold do not necessarily mean larger sales. there is a confounding variable here i.e. Unitprice

In [ ]:
# hourly purchases (busiest hour)
df['Hour'] = df['InvoiceDate'].dt.hour
df.Hour.unique()

hour = [ 6, 7, 8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
# converting the hour data type to ordinal
ordered_hour = pd.api.types.CategoricalDtype(ordered = True, categories = hour)
df['Hour'] = df['Hour'].astype(ordered_hour)
df.info()

grouping_by_hour = df.groupby(['Hour'])['Quantity'].count().sort_values(ascending = False)
print(grouping_by_hour)

# visualize the busiest hours
sns.lineplot(data = grouping_by_hour)
plt.title("Orders by Hour")
plt.show();

Note:
- From the diagram, our busiest hours are approximately 12pm and 3 pm
- The store can increase advertisements around this time to pull in more customers

In [ ]:
# daily orders (busiest day)
df['Weekday'] = df['InvoiceDate'].dt.day_name()
print(df.Weekday.unique())

day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
# converting the day data type to ordinal
ordered_day = pd.api.types.CategoricalDtype(ordered = True, categories = day)
df['Weekday'] = df['Weekday'].astype(ordered_day)
df.info()

grouping_by_day = df.groupby(['Weekday'])['Quantity'].count().sort_values(ascending = False).reset_index()
print(grouping_by_day)

# visualize the busiest days
sns.barplot(data = grouping_by_day, x = 'Weekday', y= 'Quantity', color = sns.color_palette()[0])
plt.title("Orders by Day")
plt.show()

Note:
- As imagined, our diagram did not contain data for Sunday. Friday represents our busiest day at the store.
- the store can increase its advertisements on this day to increase orders

***Multivariate Analysis***

In [ ]:
# relationship between price, sales and quantity ordered
print(total_purch_per_day)
plt.figure(figsize = (10, 5))

sns.scatterplot(x = total_purch_per_day.UnitPrice, y = total_purch_per_day.Sales, hue= total_purch_per_day.Quantity)
plt.show()

In [ ]:
print("correlation b/n price and sales: ", total_purch_per_day['UnitPrice'].corr(total_purch_per_day['Sales']))
print("correlation b/n price and quantity: ", total_purch_per_day['UnitPrice'].corr(total_purch_per_day['Quantity']))

Note:
- the impact price has on sales is not so different from the impact price has on quantity bought. A bigger price could have orders with larger quantities. Price does not really make a difference here 

### Analyze the product range

Note:
- Lets visualize some of the products we have in the store as well as top products from orders

In [ ]:
df_visualize = df.copy()
df_visualize['all'] = 'all'
fig = px.treemap(df_visualize.head(30), path=['all', "Description"], values='Quantity', 
                 color = df_visualize["Quantity"].head(30), hover_data=['Description'], color_continuous_scale='Blues')
fig.update_layout(title = 'Some Products at the Store')
fig.show()

In [ ]:
# visualize top 15 products from orders
df_network = df.copy()
df_network_first = df_network.groupby("Description").sum().sort_values("Quantity", ascending=False).reset_index()[:15]
df_network_first["ItemType"] = "Store Products"
print(df_network_first)

plt.figure(figsize = (20,20))
first_choice = nx.from_pandas_edgelist(df_network_first, source='ItemType', target="Description", edge_attr=True)
pos = nx.spring_layout(first_choice)
nx.draw_networkx_nodes(first_choice, pos, node_size=12500, node_color="lavender")
nx.draw_networkx_edges(first_choice, pos, width=2, alpha=0.5, edge_color='black')
nx.draw_networkx_labels(first_choice, pos, font_size=12, font_family='sans-serif')
plt.axis('off')
plt.grid()
plt.title('Top 15 Products', fontsize=25)
plt.show()

### Formulate and test statistical hypotheses:

***problem statement 1: How stable is sales? 
Hypotheses: can it be explained by demand variability or seasonality?***

In [ ]:
# monthly sales
monthly_sales = df.set_index('InvoiceDate').resample('M').sum().reset_index()
print(monthly_sales)

fig = px.line(data_frame = monthly_sales, x = 'InvoiceDate', y = 'Sales')
fig.update_layout(title = "Monthly Sales")
fig.show()

Note 1:

Peak sales were around November 2019. The same cannot be said for 2018. Why is that?

- could it be that the store launched around November, 2018?

- did the store introduce new products or strategies in November, 2019 which drove sales?

***not enough data to dig deeper into these hypotheses.***

Again, we see that sales is not stable over the months. Demand varies across the months and since the data covers a year's sales, unbiased conclusions can not be drawn with regards to seasonality. More data would be needed to conclude if sales is seasonal or not. 


`Conclusion: whatever strategy the store used during November 2019 (Christmas season) really drove sales.`

Note 2:

I would use the following formula to check for variability in sales:

- Coefficient of Variation (CV) = Average Sales/Standard Deviation of Sales

In [ ]:
avg_sales = df.Sales.mean()
print('Average Sales: ', avg_sales)
std_sales = df.Sales.std()
print('Standard Deviation: ', std_sales)
CV_sales = (std_sales/avg_sales)* 100
print('Coefficient of Variation: ', CV_sales)

Note:
- A high coefficient of variation depicts unstable customer demand hence the store may be faced with forecasting complexities, workload peaks and planning and distribution challenges.

`recall that the graphical distribution of sales did not show a normal distribution.`

- For this kind of distribution, a cause analysis would provide better results than the statistical approach used here.

***problem statement 2: What items should the store invest in more? Hypotheses: Can this be explained by price (affordability)***


Note:

- Categorize products using RFM segmentation and label cohorts: very fast movers, fast movers, slow movers and very slow movers

Using RFM segmentation on products made a lot of sense to me under this circumstance, in that, it encompasses all aspects (monetary, recency and frequency). It reduces bias here.

for example: segmenting products based on sales only (monetary) will have items that are not frequently bought but have high prices or sales categorized as fast movers, which would be wrong. Should the store stock up on these products, warehouse spacing would be an issue here.

RFM segmentation based on RFM score will output product categories that give the store a 'bit of everything' in terms of sales, recency and frequency.

In [ ]:
# extract year, month and day
df['InvoiceDay'] = df['InvoiceDate'].dt.year.astype(str) + "-" + df['InvoiceDate'].dt.month.astype(str) + "-" + df['InvoiceDate'].dt.day.astype(str)
print(df.head())

In [ ]:
df.drop(['Weekday', 'Hour'], axis = 1, inplace = True)
print(df.head())

In [ ]:
df['InvoiceDay'] = pd.to_datetime(df['InvoiceDay'])
df.info()

In [ ]:
 # recall the time period
print('Min : {}, Max : {}'.format(min(df.InvoiceDay), max(df.InvoiceDay)))

In [ ]:
# pin the last date
import datetime
from datetime import timedelta

snapshot_date = max(df.InvoiceDay) + datetime.timedelta(days=1)
snapshot_date

In [ ]:
# calculate RFM values
rfm = df.groupby('Description').agg({'UnitPrice': 'mean',
    'InvoiceDate' : lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo' : 'count', 
    'Sales' : 'sum'})
# rename the columns
rfm.rename(columns = {'InvoiceDate' : 'Recency', 
                      'InvoiceNo' : 'Frequency', 
                      'Sales' : 'Monetary'}, inplace = True)
rfm.head()

In [ ]:
# create labels and assign them to percentile groups 
r_labels = range(3, 0, -1)
r_groups = pd.qcut(rfm['Recency'], q = 4, labels = r_labels, duplicates='drop')
f_labels = range(1, 5)
f_groups = pd.qcut(rfm.Frequency, q = 4, labels = f_labels)
m_labels = range(1, 5)
m_groups = pd.qcut(rfm.Monetary, q = 4, labels = m_labels)

In [ ]:
# make a new column for group labels
rfm['R'] = r_groups.values
rfm['F'] = f_groups.values
rfm['M'] = m_groups.values
# sum up the three columns
rfm['RFM_Segment'] = rfm.apply(lambda x: str(x['R']) + str(x['F']) + str(x['M']), axis = 1)
rfm['RFM_Score'] = rfm[['R', 'F', 'M']].sum(axis = 1)
rfm.head()

In [ ]:
# calculate average values for each RFM
rfm_agg = rfm.groupby('RFM_Score').agg({
    'Recency' : 'mean',
    'Frequency' : 'mean',
    'Monetary' : ['mean', 'count']
})
rfm_agg.round(1).head()

In [ ]:
# assign labels from total score
score_labels = ['Very_slow_movers', 'Slow_movers', 'Fast_movers', 'Very_fast_movers']
score_groups = pd.qcut(rfm.RFM_Score, q = 4, labels = score_labels)
rfm['RFM_Level'] = score_groups.values
rfm.head()

In [ ]:
rfm.sort_values('RFM_Score', ascending = False)

***problem statement 3: Does price affect sellability?***

This would be checked on the categorized products

In [ ]:
very_fast_movers = rfm.loc[rfm['RFM_Level'] == 'Very_fast_movers'].sort_values('Monetary', ascending = False).reset_index()
print(very_fast_movers.head())

#plotting top 20 very_fast_movers against average price
ax = very_fast_movers[:20].plot('Description', 'Monetary', kind = 'bar', color = 'g', figsize = (10, 8))
ax1 = ax.twinx()
very_fast_movers[:20].plot('Description', 'UnitPrice', ax = ax1, color = 'r')

plt.title('Very Fast Moving Items vs Avg Price')
ax.set_xlabel('Item Description')
ax.set_ylabel('Sales', color = 'g')
ax1.set_ylabel('Avg Price', color = 'r')
ax.get_legend().remove()
plt.show()

Note:

- 'REGENCY CAKESTAND 3 TIER': most purchased yet most returned store item

In [ ]:
avg_sales = very_fast_movers.Monetary.mean()
print('Average Sales for Very Fast movers: ', avg_sales)
std_sales = very_fast_movers.Monetary.std()
print('Standard Deviation for Very Fast movers: ', std_sales)
CV_sales = (std_sales/avg_sales)* 100
print('Coefficient of Variation: ', CV_sales)

In [ ]:
very_fast_movers['Monetary'].plot.hist(bins = 50, alpha = 0.5);

Note :

- prices vary much across the items in this category. Prices fall below 10 except for 'REGENCY CAKESTAND 3 TIER' which has an average price above 10.

- Also, the very fast-movers category has a varied demand with a coefficient of variation at 121%. There is a high variability in the data distribution. Although not a normal distribution, majority of the orders or demand are in the stable area. The store would have some planning and distribution challenges with this category.

***fast-moving items***

In [ ]:
fast_movers = rfm.loc[rfm['RFM_Level'] == 'Fast_movers'].sort_values('Monetary', ascending = False).reset_index()
print(fast_movers.head())

#plotting top 20 fast-movers against average price
ax = fast_movers[:20].plot('Description', 'Monetary', kind = 'bar', color = 'g', figsize = (10, 8))
ax1 = ax.twinx()
fast_movers[:20].plot('Description', 'UnitPrice', ax = ax1, color = 'r')

plt.title('Fast movers vs Avg Price')
ax.set_xlabel('Item Description')
ax.set_ylabel('Sales', color = 'g')
ax1.set_ylabel('Avg Price', color = 'r')
ax.get_legend().remove()
plt.show()

In [ ]:
avg_sales = fast_movers.Monetary.mean()
print('Average fast movers Sales: ', avg_sales)
std_sales = fast_movers.Monetary.std()
print('Standard Deviation of fast movers: ', std_sales)
CV_sales = (std_sales/avg_sales)*100
print('Coefficient of Variation: ', CV_sales)

In [ ]:
fast_movers['Monetary'].plot.hist(bins = 100, alpha = 0.5)

Note:

- We see that majority of the item prices fall below 10

- this category has a low coefficient of variation (104%) compared to the very fast movers. this category has a varied distribution of data. planning and distribution challenges would still be faced here.

***Slow movers***

In [ ]:
slow_movers = rfm.loc[rfm['RFM_Level'] == 'Slow_movers'].sort_values('Monetary', ascending = False).reset_index()
print(slow_movers.head())

#plotting top 20 slow-movers against average price
ax = slow_movers[:20].plot('Description', 'Monetary', kind = 'bar', color = 'g', figsize = (10, 8))
ax1 = ax.twinx()
slow_movers[:20].plot('Description', 'UnitPrice', ax = ax1, color = 'r')

plt.title('Items with slow-sales vs Avg Price')
ax.set_xlabel('Item Description')
ax.set_ylabel('Sales', color = 'g')
ax1.set_ylabel('Avg Price', color = 'r')
ax.get_legend().remove()
plt.show()

In [ ]:
mean_sales = slow_movers.Monetary.mean()
print('Average sales for Slow movers: ', mean_sales)
std_sales = slow_movers.Monetary.std()
print('Standard Deviation for Slow movers: ', std_sales)
CV_sales = (std_sales/mean_sales)*100
print('Coefficient of Variation: ', CV_sales)

In [ ]:
slow_movers['Monetary'].plot.hist(bins = 20, alpha = 0.5);

Note:

-  This category has store items with prices above 100 and store items with prices slightly above 0.

- it also has a higher coefficient of variation at 267%

***Very Slow movers***

In [ ]:
very_slow_movers = rfm.loc[rfm['RFM_Level'] == 'Very_slow_movers'].sort_values('Monetary', ascending = False).reset_index()
print(very_slow_movers.head())

#plotting top 20 very_slow-movers against average price
ax = very_slow_movers[:20].plot('Description', 'Monetary', kind = 'bar', color = 'g', figsize = (10, 8))
ax1 = ax.twinx()
very_slow_movers[:20].plot('Description', 'UnitPrice', ax = ax1, color = 'r')

plt.title('Items with very slow-sales vs Avg Price')
ax.set_xlabel('Item Description')
ax.set_ylabel('Sales', color = 'g')
ax1.set_ylabel('Avg Price', color = 'r')
ax.get_legend().remove()
plt.show()

In [ ]:
mean_sales = very_slow_movers.Monetary.mean()
print('Average of Very Slow movers Sales: ', mean_sales)
std_sales = very_slow_movers.Monetary.std()
print('Standard Deviation of Very Slow movers: ', std_sales)
CV_sales = (std_sales/mean_sales)*100
print('Coefficient of Variation: ', CV_sales)

In [ ]:
very_slow_movers['Monetary'].plot.hist(bins = 100, alpha = 0.5);

Note:

- Like the very fast movers, this category has prices below 10 except for 'Vintage Post Office Cabinet' which has a high price, above 60.


`Conclusions on the categorization done:`

- the store is a wholesale vendor because it has a very varied sales distribution. Or it probably sells durable items which have a varied demand, like gift items.

- The store may have to deal with forecasting challenges as well as planning and distribution challenges for all categories.


- Albeit, a cause analysis will provide much more informed findings than the statistical approaches used here.

***problem statement 4:  What products are most often sold together?
Hypotheses: association rules***

In [ ]:
# using apriori
# summing up multiple products within the same order
df.head()
df_grouped = df.groupby(['InvoiceNo','Description']).sum()['Quantity']
df_grouped

In [ ]:
# unstack the grouped dataframe
df_basket = df_grouped.unstack().reset_index().fillna(0).set_index('InvoiceNo')

# set values to 0s and 1s
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
    
basket_sets = df_basket.applymap(encode_units)
basket_sets.head()

In [ ]:
# popular products in the store
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True).sort_values(by=['support'], ascending=False)
frequent_itemsets.head(15)

Note:
- The support column indicates how frequently the item appears in the dataset i.e. how popular a product is in the shop

This is telling us that 'WHITE HANGING HEART T-LIGHT HOLDER', 'JUMBO BAG RED RETROSPOT', 'REGENCY CAKESTAND 3 TIER' are the most popular items in this shop. 

#### Use the function, “association_rules” from, “mlxtend” to create the dataframe from, “frequent_itemsets” using the arguments metric, “lift”.

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift")
rules.head()

Note:

- Lift is a metric to measure the ratio of the confidence of products occurring together if they were statistically independent. E.g. how likely is another product purchased when purchasing a product, while controlling how popular the other product is. A lift score that is close to 1 indicates that the antecedent and the consequent are independent and occurrence of antecedent has no impact on occurrence of consequent. A Lift score that is greater than 1 indicates that the antecedent and consequent are dependent to each other, and the occurrence of antecedent has a positive impact on occurrence of consequent. A lift score that is smaller than 1 indicates that the antecedent and the consequent are substitute each other that means the existence of antecedent has a negative impact to consequent or visa versa.


- A lift ratio larger than 1.0 implies that the relationship between the two products is more significant than would be expected if the two sets were independent. The larger the lift ratio, the more significant the association. “confidence” is how often the rule has been found to be true. In order words, how reliable this rule is as a percentage.


- Therefore, what we are looking for are rules where the “lift” and “confidence” is highest.

In [ ]:
rules[(rules['lift'] > 1) & (rules['confidence'] >= 0.25) ]

Note:

- The “antecedent” is the product being purchased and the “consequent” is the product that is often purchased as well with their associated likelihood.

- This is telling us that in this shop if someone buys a “ROSES REGENCY TEACUP AND SAUCER", they are likely to buy “GREEN REGENCY TEACUP AND SAUCER”  as well.

### References

During my research on the topic, the following websites enlightened me on some of the questions a data analyst should aim to answer when conducting product range analysis and how to go about some of the tasks.

https://towardsdatascience.com/product-segmentation-for-retail-with-python-c85cc0930f9a (for general research purposes)

https://towardsdatascience.com/shop-order-analysis-in-python-ff13615404e0 (for general research purposes)

https://medium.com/swlh/product-sales-analysis-using-python-863b29026957 (for general research purposes)

https://sunscrapers.com/blog/sales-data-science-a-step-by-step-guide-to-competitor-analysis-using-python/ (for general research purposes)

https://stackoverflow.com/questions/60558920/looking-for-positive-values-that-match-negative-values-within-a-column-pandas (assisted me on how to get both the original and returned goods in a dataframe)

https://stackoverflow.com/questions/46063379/pandas-secondary-axis (assisted me on how to plot on a secondary axis)

https://github.com/finnqiao/cohort_online_retail/blob/master/ukretail_cohort.ipynb

https://www.kdnuggets.com/2019/05/golden-goose-cohort-analysis.html (assisted me on how to apply RFM segmentation which I used on the products instead of customers)